In [1]:
import pandas as pd
import numpy as np

Goal: create single-node version of Magude-Sede-Facazissa spatial sims, for e2e workflow

In [5]:
df_lookup = pd.read_csv("C:/Users/joshsu/Dropbox (IDM)/Malaria Team Folder/projects/mz_magude/inputs/csvs/reference/grid_lookup.csv")

In [7]:
df_lookup["catchment"].unique()

array(['Mapulanguene', 'Panjane-Caputine', 'Mahel', 'Chichuco', 'Moine',
       'Magude-Sede-Facazissa', 'Chicutso', 'Motaze'], dtype=object)

In [8]:
df_lookup = df_lookup[df_lookup["catchment"] == "Magude-Sede-Facazissa"].reset_index(drop=True)

In [9]:
df_lookup

,grid_cell,lon,lat,bairro,bairro_name,catchment,pop
0,543,32.524138,-25.055216,60,Chissanguane,Magude-Sede-Facazissa,1.0
1,544,32.524138,-25.046233,60,Chissanguane,Magude-Sede-Facazissa,6.0
2,579,32.534158,-25.046233,60,Chissanguane,Magude-Sede-Facazissa,6.0
3,580,32.534158,-25.037250,60,Chissanguane,Magude-Sede-Facazissa,2.0
4,581,32.534158,-25.028267,60,Chissanguane,Magude-Sede-Facazissa,1.0
...,...,...,...,...,...,...,...
326,1169,32.834752,-24.965385,6,Facazisse,Magude-Sede-Facazissa,2.0
327,1187,32.844772,-24.974368,6,Facazisse,Magude-Sede-Facazissa,10.0
328,1188,32.844772,-24.965385,6,Facazisse,Magude-Sede-Facazissa,10.0
329,1205,32.854791,-24.974368,6,Facazisse,Magude-Sede-Facazissa,3.0


In [10]:
df_itn = pd.read_csv("C:/Users/joshsu/Dropbox (IDM)/Malaria Team Folder/projects/mz_magude/inputs/csvs/interventions/grid_all_itn_events.csv")

In [18]:
df_itn = pd.merge(df_itn, df_lookup[["grid_cell", "pop"]], on="grid_cell", how="inner")

In [19]:
df_itn

,event,fulldate,cov_all,age_cov,min_season_cov,fast_fraction,grid_cell,discard_type_flag,killing_initial,killing_decay_constant,blocking_initial,blocking_decay_constant,max_season_cov_day,pop
0,1,2014-02-01,0.40,0.9,0.58,0.6,544,0,0.70,781,0.60,2046,40,6.0
1,2,2015-05-01,0.50,0.9,0.58,0.0,544,1,0.39,781,0.48,2046,40,6.0
2,3,2015-09-01,0.40,0.9,0.58,0.0,544,1,0.34,781,0.45,2046,40,6.0
3,4,2017-12-01,0.75,0.9,0.58,0.0,544,1,0.70,781,0.60,2046,40,6.0
4,1,2014-02-01,0.40,0.9,0.58,0.6,579,0,0.70,781,0.60,2046,40,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179,4,2017-12-01,0.75,0.9,0.58,0.0,1205,1,0.70,781,0.60,2046,40,3.0
1180,1,2014-02-01,0.40,0.9,0.58,0.6,1206,0,0.70,781,0.60,2046,40,37.0
1181,2,2015-05-01,0.50,0.9,0.58,0.0,1206,1,0.39,781,0.48,2046,40,37.0
1182,3,2015-09-01,0.75,0.9,0.58,0.0,1206,1,0.34,781,0.45,2046,40,37.0


In [17]:
for i, sdf in df_itn.groupby("event"):
    print(sdf["fulldate"].unique())

['2014-02-01']
['2015-05-01']
['2015-09-01']
['2017-12-01']


In [23]:
def weighted_average(group, col_name, weight_column="pop"):
    return (group[col_name] * group[weight_column]).sum() / group[weight_column].sum()


In [22]:
from functools import partial

In [26]:
df_itn.groupby("event").apply(partial(weighted_average, col_name="cov_all"))

event
1    0.400000
2    0.500000
3    0.704861
4    0.750000
dtype: float64

In [27]:
df_itn.groupby("event").apply(partial(weighted_average, col_name="age_cov"))

event
1    0.9
2    0.9
3    0.9
4    0.9
dtype: float64

In [28]:
df_itn.groupby("event").apply(partial(weighted_average, col_name="min_season_cov"))

event
1    0.58
2    0.58
3    0.58
4    0.58
dtype: float64

In [32]:
df_itn.groupby("event").apply(partial(weighted_average, col_name="blocking_initial"))

event
1    0.60
2    0.48
3    0.45
4    0.60
dtype: float64